# Strong business tester

This notebook tries to test if a business is strong based on some financial parameters:

* Strong free cash flow
* Strong balance sheet

In [1]:
from yahooquery import Ticker

## Has strong business?

In [2]:
def non_decreasing(L):
    return all(x<=y for x, y in zip(L, L[1:]))

def non_increasing(L):
    return all(x>=y for x, y in zip(L, L[1:]))

def monotonic(L):
    return non_decreasing(L) or non_increasing(L)

def strictly_increasing(L):
    return all(x<y for x, y in zip(L, L[1:]))

def strictly_decreasing(L):
    return all(x>y for x, y in zip(L, L[1:]))

def strictly_monotonic(L):
    return strictly_increasing(L) or strictly_decreasing(L)

In [3]:
def has_consecutive_positive_fcf(ticker):
    cash_flow = ticker.cash_flow(frequency='Annual')
    free_cash_flow = cash_flow[['asOfDate', 'FreeCashFlow']]
    free_cash_flow = free_cash_flow.set_index('asOfDate')
    fcf = free_cash_flow.to_dict()['FreeCashFlow']
    return all([v > 0 for v in fcf.values()])

In [4]:
def has_consistently_low_debt_ratios(debt_equity_ratio_values):
    return all([v < 0.3 for v in debt_equity_ratio_values])


def has_strong_balance_sheet(ticker):
    balance_sheet = ticker.balance_sheet(frequency='Quarterly')
    balance_sheet['TotalDebt/CommonStockEquity']=balance_sheet['TotalDebt']/balance_sheet['CommonStockEquity']
    debt_equity_ratio = balance_sheet[['asOfDate', 'TotalDebt/CommonStockEquity']]
    debt_equity_ratio = debt_equity_ratio.set_index('asOfDate').to_dict()['TotalDebt/CommonStockEquity']
    debt_equity_ratio_values = list(debt_equity_ratio.values())

    if not has_consistently_low_debt_ratios(debt_equity_ratio_values):
        print(f"{ticker} doesn't have consistently low debt ratios")
        return False
    
    if not strictly_decreasing(debt_equity_ratio_values):
        print(f"{ticker} doesn't have strictly decreasing debt equity ratio")
        return False
    
    return True

In [5]:
def has_strong_business(ticker):
    if not has_consecutive_positive_fcf(ticker):
        print(f"{ticker} doesn't have consecutive positive fcf")
        return False
    
    if not has_strong_balance_sheet(ticker):
        print(f"{ticker} doesn't have strong balance sheet")
        return False
    
    return True

In [6]:
has_strong_business(Ticker('TSLA'))

True

In [7]:
has_strong_business(Ticker('LCID'))

<yahooquery.ticker.Ticker object at 0x11c470bd0> doesn't have consecutive positive fcf


False

In [8]:
has_strong_business(Ticker('AMZN'))

<yahooquery.ticker.Ticker object at 0x11c4897d0> doesn't have consecutive positive fcf


False

In [9]:
has_strong_business(Ticker('BABA'))

<yahooquery.ticker.Ticker object at 0x11c49da90> doesn't have strictly decreasing debt equity ratio
<yahooquery.ticker.Ticker object at 0x11c49da90> doesn't have strong balance sheet


False